In [1]:
from imports import *
from data_generator import *
from models import *

#shouldn't need on a local computer?
from utils import limit_mem

#Module to save and load models
import h5py
import netCDF4
import numpy as np

from keras.models import Sequential
from keras.layers import *
import time
#added by Griffin
import xarray as xr


from tensorflow.python.ops.parallel_for.gradients import batch_jacobian
from tensorflow.python.ops.parallel_for.gradients import jacobian

Using TensorFlow backend.


In [2]:
output_vector = 65
input_vector = 64
print('Starting')


#If running on the GPU or GPU-shared partition uncomment
limit_mem()


DATADIR = '/DFS-L/DATA/pritchard/gmooers/RG_Paper/RG_DATA/Preprocessed_Data/One_Month_July/'



valid_gen = DataGenerator(
    data_dir=DATADIR, 
    feature_fn='full_physics_essentials_valid_month02_features.nc',
    target_fn='full_physics_essentials_valid_month02_targets.nc',
    batch_size=512,
    norm_fn='full_physics_essentials_train_month01_norm.nc',  # SAME NORMALIZATION FILE!
    fsub='feature_means', 
    fdiv='feature_stds', 
    tmult='target_conv',
    shuffle=False,
)


fsub = valid_gen.feature_norms[0]
fdiv = valid_gen.feature_norms[1]
tsub = valid_gen.target_norms[0]
tdiv = valid_gen.target_norms[1]


model = tf.keras.models.load_model('/fast/gmooers/Models/ContinentModels/second_round_model.h5')

path_to_file = '/DFS-L/DATA/pritchard/gmooers/RG_Paper/RG_DATA/Preprocessed_Data/One_Month_July/full_physics_essentials_valid_month02_features.nc'
real_ds = xr.open_dataset(path_to_file)
features = real_ds.features[:, :].values
print(features.shape)


path_to_file = '/DFS-L/DATA/pritchard/gmooers/RG_Paper/RG_DATA/Preprocessed_Data/One_Month_July/full_physics_essentials_valid_month02_targets.nc'
real_ds = xr.open_dataset(path_to_file)
targets = real_ds.targets[:, :].values
print('files imported')
model_data = np.zeros(shape=(len(features), output_vector))
model_data[:,:] = np.nan

Starting


Generator will have 41126400 samples in 80325 batches
Features have shape 64; targets have shape 65
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
(41126400, 64)
files imported


https://stackoverflow.com/questions/53995921/getting-keras-predictions-as-a-tensor-graph-for-use-in-tensorflow

https://stackoverflow.com/questions/44274701/make-predictions-using-a-tensorflow-graph-from-a-keras-model

In [11]:
#model = tf.keras.models.load_model("model.h5")
model_output = model.output
model_input = model.input
new_tensor_ph = tf.placeholder(tf.float32, [None, 1])
new_output = tf.multiply(model_output, new_tensor_ph)
#x_tf = tf.convert_to_tensor(np.expand_dims(features[0,:],axis=0))
f = features[0]
f=f.reshape(-1,1)
x = np.reshape(f, (1,64))
f = (x-fsub)/fdiv
sess = tf.keras.backend.get_session() 
f = tf.convert_to_tensor(f)
prediction = sess.run(new_output, feed_dict={model.input:f,new_tensor_ph :[[4]]})
print(prediction)
## This works without error

TypeError: The value of a feed cannot be a tf.Tensor object. Acceptable feed values include Python scalars, strings, lists, numpy ndarrays, or TensorHandles. For reference, the tensor object was Tensor("Const_5:0", shape=(1, 64), dtype=float32) which was passed to the feed with key Tensor("input_1:0", shape=(?, 64), dtype=float32).

In [ ]:
something = model(x_init_value)
print(something)

In [ ]:
sess = tf.InteractiveSession()
f = features[0]
f=f.reshape(-1,1)
x = np.reshape(f, (1,64))
x_init_value = (x-fsub)/fdiv
x_placeholder =  tf.placeholder(tf.float32)
x_var = tf.Variable(x_init_value, dtype=tf.float32)

# Check calling my_model
assign_step = tf.assign(x_var, x_placeholder)
sess.run(assign_step, feed_dict={x_placeholder: x_init_value})
model_output = model(x_var) # This simple step is all I wanted!
sess.run(model_output) # This outputs my_model's predicted value for input x_init_value

In [ ]:
def get_jacobian(x, my_model):
    sess = tf.keras.backend.get_session()
    jac = jacobian(my_model.output, my_model.input)
    J = sess.run(jac, feed_dict={my_model.input: x.astype(np.float32)[None]})
    return J.squeeze()

def get_batch_jacobian(x, model):
    sess = tf.keras.backend.get_session()
    jac = batch_jacobian(model.output, model.input)
    J = sess.run(jac, feed_dict={model.input: x.astype(np.float32)})
    return J.squeeze()

get_jacobian(f, model)

In [27]:
def bootleg_jacobian(A, b):
    dotProduct = A.dot(b)
    ans = np.zeros((A.shape[0], A.shape[1], dotProduct.shape[0], dotProduct.shape[1]))
    for l in range(A.shape[0]):
        i = 0
        for k in range(A.shape[1]):
            ans[l][k][l][:] = b[i][:]
            i = i + 1
    return ans

def vect_jacobian(A,b):

    dotProduct = A.dot(b)
    ans = np.zeros((A.shape[0], A.shape[1], dotProduct.shape[0], dotProduct.shape[1]))
    L = list(range(A.shape[0]))
    ans[L,:,L,:] = b
    return ans

#r = vect_jacobian(A,b)

In [24]:
f = features[0]
f=f.reshape(-1,1)
x = np.reshape(f, (1,64))
f = (x-fsub)/fdiv
p = model.predict_on_batch(f)

In [21]:
print(type(f))
print(type(p))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [28]:
vect_jacobian(p, f)

ValueError: shapes (1,65) and (1,64) not aligned: 65 (dim 1) != 1 (dim 0)

In [5]:
J = np.zeros((65, 64))
#x_tf = tf.convert_to_tensor(np.expand_dims(features[0,:],axis=0))
#f = (x_tf-fsub)/fdiv
with tf.GradientTape() as tape:
    #x_tf = tf.convert_to_tensor(np.expand_dims(features[0,:],axis=0))
    x_tf = tf.convert_to_tensor(np.expand_dims(features[0,:],axis=0))
    tape.watch(x_tf)
    f = (x_tf-fsub)/fdiv
    print(type(f))
   # p = tf.convert_to_tensor(model.predict_on_batch(f))
    #p = model.predict_on_batch(f)
    p = model.predict(f, steps=1)
    print(type(p))
    #print(fgdfdfggf)
    #print(f.shape)
    #pred = (p/tdiv)+tsub
    #print(type(p))
    #print(p.shape)
    #pred = tf.convert_to_tensor(pred)
    #print(type(pred))
    #out_i = pred[0, 0]
    #print(out_i*10000,0)

grads = tape.jacobian(p, f)
print(grads)
#J[i, :] = grads/(fdiv*cf)

<class 'tensorflow.python.framework.ops.Tensor'>
<class 'numpy.ndarray'>
None


In [ ]:
J=jacobian(p,f)
print(J)

In [ ]:
print(J)

In [ ]:
trainingExample = np.random.random((1,8))
sess = tf.InteractiveSession()
sess.run(tf.initialize_all_variables())
evaluated_gradients = sess.run(gradients,feed_dict={model.input:trainingExample})

In [31]:
def jacobian_tensorflow(x):    
    jacobian_matrix = []
    for m in range(65):
        print(m)
        # We iterate over the M elements of the output vector
        grad_func = tf.gradients(model.output[:, m], model.input)
        gradients = sess.run(grad_func, feed_dict={model.input: x.reshape((1, x.size))})
        jacobian_matrix.append(gradients[0][0,:])
    return np.array(jacobian_matrix)

In [33]:
jaboc = jacobian_tensorflow(features[0])
print(jaboc.shape)
print(jaboc)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
(65, 64)
[[-7.67039601e-03  5.11548575e-03 -3.89819936e-04 ...  6.52137026e-02
  -1.61790904e-02  2.91623324e-02]
 [-5.43078873e-03  2.78798304e-03 -7.36124639e-05 ...  5.11873886e-02
  -1.55727854e-02  2.63065957e-02]
 [ 1.37523958e-03 -2.88796000e-04 -9.33522708e-04 ...  3.48145068e-02
   4.39121388e-03  1.24364896e-02]
 ...
 [-8.97872611e-04 -5.32480329e-03 -9.10913339e-04 ...  1.28090149e-03
   1.45285027e-02  2.63195448e-02]
 [-1.12135269e-04  7.58875976e-04 -1.84966868e-03 ... -1.37105742e-02
   1.26841869e-02  5.02925599e-04]
 [ 1.58913119e-03  1.26594165e-02 -1.26897758e-02 ... -6.26130495e-03
   2.37920275e-03 -2.72885915e-02]]
